In [1]:
import getpass
import os
import sys
import time

import numpy as np
import pykeen
from pykeen.pipeline import pipeline
from pykeen.triples.leakage import Sealant
from pykeen.triples import TriplesFactory

In [2]:
print(sys.version)

3.8.3 (default, May 27 2020, 20:54:22) 
[Clang 11.0.3 (clang-1103.0.32.59)]


In [3]:
print(time.asctime())

Fri Jun 26 23:22:11 2020


In [4]:
print(getpass.getuser())

cthoyt


In [5]:
print(pykeen.get_version(with_git_hash=True))

1.0.1-dev-8840f3a8


In [6]:
ras_triples_path = 'ras_machine_triples.tsv'
missing_ras_triples = not os.path.exists(ras_triples_path)
missing_ras_triples

False

In [7]:
if missing_ras_triples:
    import pybel
    graph = pybel.from_emmaa('rasmachine')

In [8]:
if missing_ras_triples:
    graph.summarize()

In [9]:
if missing_ras_triples:
    import pybel.io.tsv
    triples = np.array(pybel.io.tsv.api.get_triples(graph))
    np.savetxt(ras_triples_path, triples, fmt='%s', delimiter='\t')
else:
    triples = np.loadtxt(ras_triples_path, dtype=str, delimiter='\t')

In [10]:
tf = TriplesFactory(triples=triples)

In [11]:
training, testing = tf.split(0.8, random_state=1234)

In [12]:
results = pipeline(
    training_triples_factory=training,
    testing_triples_factory=testing,
    model='TransE',
    training_kwargs=dict(num_epochs=5),
    random_seed=1235,
    device='cpu',
)

IndexError: index out of range in self